In [3]:
import pandas as pd

In [8]:
airbnb = pd.read_csv('listings.csv')
#airbnb
## Data Cleaning of CSV
df = pd.read_csv("listings.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,60,2,178,12/15/19,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,7/14/20,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,3/8/20,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,10/23/19,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2/14/20,0.61,5,168


In [9]:
df=airbnb.drop(['neighbourhood_group'], axis=1)

KeyError: "['neighbourhood_group'] not found in axis"

In [10]:
df

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,Hyde Park,41.78790,-87.58780,Private room,60,2,178,12/15/19,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,7/14/20,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,3/8/20,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,10/23/19,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2/14/20,0.61,5,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,45351578,Spacious Modern Vibe Avondale/Logan -Entire Apt,77382816,Edmund,Avondale,41.94060,-87.72281,Entire home/apt,67,1,2,9/20/20,2.00,1,307
5122,45368527,Private Bedroom in Chicago Near Lincoln Square...,128265803,Victor,North Park,41.99222,-87.71659,Private room,24,1,2,9/20/20,2.00,4,14
5126,45386114,BEST PRICE for a studio in Wrigleyville-Boystown,324740940,Andrea,Uptown,41.95513,-87.65095,Entire home/apt,97,1,2,9/19/20,2.00,4,81
5145,45433310,Luxury King size (78*82) bedroom with free par...,111872967,Julie,Near West Side,41.87195,-87.66456,Private room,54,1,1,9/18/20,1.00,4,359


In [11]:
df.nlargest(10, ['number_of_reviews'])

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
686,10069247,Shared Artist Loft Own Private Room,51668477,Michael,Near North Side,41.88924,-87.63032,Private room,102,1,632,9/20/20,10.97,8,329
49,464581,"Large, Private Logan Square Studio",2308792,Jonathan,Logan Square,41.92451,-87.69385,Entire home/apt,60,2,629,9/17/20,6.39,1,127
181,2570620,"Basement Studio w/Free St. Parking, Humboldt Park",1214607,Coral,Humboldt Park,41.89826,-87.70336,Entire home/apt,45,1,552,9/19/20,7.16,2,147
95,1171860,"Lincoln Park Studio, Great Value!",2658212,James,Lincoln Park,41.92628,-87.64026,Entire home/apt,99,30,541,7/29/20,6.16,4,96
419,6504170,West Town Traveler's Lodge,4591141,Moudi,West Town,41.90064,-87.67056,Entire home/apt,106,7,511,6/10/20,7.90,2,23
26,189821,"Best in Chicago, private, amazing garden space",899757,Meighan,Logan Square,41.92918,-87.70219,Entire home/apt,201,3,506,9/12/20,4.61,1,134
96,1183249,"Boystown Studio, Outstanding Value!",2658212,James,Lake View,41.93950,-87.64483,Entire home/apt,99,26,504,7/24/20,5.67,4,31
44,350347,"Luxury Chicago Loft, 1 blk to train w/ 2 pkg s...",845242,Jay & Elisa,Lake View,41.93986,-87.65473,Entire home/apt,128,3,499,9/6/20,4.79,1,313
214,3061036,Private Room-sunny loft. McCormick place/down...,1876022,Leslie,Near South Side,41.85298,-87.62761,Private room,85,1,499,9/13/20,6.46,1,152
618,8706915,"Polk Street Coach House Apartment, Little Ital...",43282155,Ken,Near West Side,41.87186,-87.66469,Entire home/apt,98,2,491,9/13/20,8.38,1,279


NameError: name 'pd' is not defined